In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection
nltk.download('punkt')

from transformers import BertTokenizer

##########################################################

import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

[nltk_data] Downloading package punkt to /home/moonstar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2023-06-26 21:06:06.488224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 21:06:07.266288: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:/home/moonstar/.mujoco/mujoco210/bin:/usr/lib/nvidia
2023-06-26 21:06:07.266386: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot

In [2]:
df_train = pd.read_csv('open/train.csv', delimiter=',', encoding = "utf8")
print(len(df_train))

# NaN값 삭제
df_train.dropna(inplace=True)
print(len(df_train))

print(df_train['facts'][0])
df_train.head()

2478
2478
On June 27, 1962, Phil St. Amant, a candidate for public office, made a television speech in Baton Rouge, Louisiana.  During this speech, St. Amant accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union.  Finally, St. Amant implicated Herman Thompson, an East Baton Rouge deputy sheriff, in a scheme to move money between the Teamsters Union and St. Amant’s political opponent. 
Thompson successfully sued St. Amant for defamation.  Louisiana’s First Circuit Court of Appeals reversed, holding that Thompson did not show St. Amant acted with “malice.”  Thompson then appealed to the Supreme Court of Louisiana.  That court held that, although public figures forfeit some of their First Amendment protection from defamation, St. Amant accused Thompson of a crime with utter disregard of whether the remarks were true.  Finally, that court held that the First Amendment protects uninhibited, robust debate, 

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [3]:
sentences = []
first_people = []
second_people = []
winners = []


for j in range(len(df_train)):

    first_party = df_train['first_party'][j]
    second_party = df_train['second_party'][j]
    # print(first_party)
    # print(second_party)

    first_party_tokens = nltk.tokenize.word_tokenize(first_party)
    second_party_tokens = nltk.tokenize.word_tokenize(second_party)
    # print(first_party_tokens)
    # print(second_party_tokens)


    fact = df_train['facts'][j]
    fact_tokens = nltk.tokenize.word_tokenize(fact)

    first = 'tony'
    second = 'jack'

    first_people.append(first)
    # first_people.append(second)
    second_people.append(second)
    # second_people.append(first)

    win = df_train['first_party_winner'][j]
    winners.append(win)
    # winners.append((win*-1)+1)

    name_tmp = []
    new = []
    new2 = []

    new_sentence1 = ''
    new_sentence2 = ''

    for i in range(len(fact_tokens)):
        token = fact_tokens[i]
        token2 = fact_tokens[i]
        plus = True

        if token in first_party_tokens:
            if token == first_party_tokens[-1]:
                token = first 
                token2 = second

            else:
                token = ''
                plus = False 

        if token in second_party_tokens:
            if token == second_party_tokens[-1]:
                token = second 
                token2 = first 

            else:
                token = ''
                plus = False

        if plus == True:
            new.append(token)
            # new2.append(token2)

            new_sentence1 += token + ' '
            # new_sentence2 += token2 + ' '


    # print(new)
    # print(new2)

    # print(new_sentence1)
    # print(new_sentence2)

    sentences.append(new_sentence1)
    # sentences.append(new_sentence2)


In [4]:
# sentences = []
# first_people = []
# second_people = []
# winners = []

# print(sentences[:5])
print(first_people[:6])
print(second_people[:6])
print(winners[:20])

['tony', 'tony', 'tony', 'tony', 'tony', 'tony']
['jack', 'jack', 'jack', 'jack', 'jack', 'jack']
[1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]


In [5]:
first_party_data_frame = pd.DataFrame(first_people, columns=['first_party'])
second_party_data_frame = pd.DataFrame(second_people, columns=['second_party'])
facts_data_frame = pd.DataFrame(sentences, columns=['facts'])
winners_data_frame = pd.DataFrame(winners, columns=['first_party_winner'])

train_data_frame = pd.concat([first_party_data_frame, second_party_data_frame, facts_data_frame, winners_data_frame],axis=1,join='inner')
train_data_frame

,first_party,second_party,facts,first_party_winner
0,tony,jack,"On June 27 , 1962 , tony , a candidate for pub...",1
1,tony,jack,Ramon Nelson was riding his bike when he suffe...,0
2,tony,jack,An Alabama state court convicted tony of murde...,1
3,tony,jack,Victor tony was convicted in state court on ev...,0
4,tony,jack,"On April 24 , 1953 in Selma , jack , an intrud...",1
...,...,...,...,...
2473,tony,jack,Congress amended the Clean Air Act through the...,1
2474,tony,jack,Inc. an investment fund purchased approximatel...,1
2475,tony,jack,"In 1992 , the District Court sentenced Manuel ...",0
2476,tony,jack,"On March 8 , 1996 , Enrico jack , a lawful per...",0


In [6]:
train_data_frame = train_data_frame[:100]
train_data_frame

,first_party,second_party,facts,first_party_winner
0,tony,jack,"On June 27 , 1962 , tony , a candidate for pub...",1
1,tony,jack,Ramon Nelson was riding his bike when he suffe...,0
2,tony,jack,An Alabama state court convicted tony of murde...,1
3,tony,jack,Victor tony was convicted in state court on ev...,0
4,tony,jack,"On April 24 , 1953 in Selma , jack , an intrud...",1
...,...,...,...,...
95,tony,jack,"In 1982 , when Congress extended Medicare to f...",1
96,tony,jack,Robert jack was indicted on charges of aggrava...,1
97,tony,jack,These two consolidated cases involve the Fair ...,1
98,tony,jack,tony was convicted and sentenced to death for ...,1


In [7]:
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in train_data_frame.facts]
document_bert[:5]

['[CLS] On June 27 , 1962 , tony , a candidate for public office , made a television speech in Baton Rouge , Louisiana . During this speech , tony accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union . Finally , tony implicated jack , an East Baton Rouge deputy sheriff , in a scheme to move money between the Teamsters Union and tony ’ s political opponent . jack successfully sued tony for defamation . Louisiana ’ s First Circuit Court of Appeals reversed , holding that jack did not show tony acted with “ malice. ” jack then appealed to the Supreme Court of Louisiana . That court held that , although public figures forfeit some of their First Amendment protection from defamation , tony accused jack of a crime with utter disregard of whether the remarks were true . Finally , that court held that the First Amendment protects uninhibited , robust debate , rather than an open season to shoot down the good

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(len(tokenized_texts[0]))
print(tokenized_texts[0])

230
['[CLS]', 'On', 'June', '27', ',', '1962', ',', 'ton', '##y', ',', 'a', 'candidate', 'for', 'public', 'office', ',', 'made', 'a', 'television', 'speech', 'in', 'Bat', '##on', 'Rouge', ',', 'Louisiana', '.', 'During', 'this', 'speech', ',', 'ton', '##y', 'accused', 'his', 'political', 'opponent', 'of', 'being', 'a', 'Communist', 'and', 'of', 'being', 'involved', 'in', 'criminal', 'activities', 'with', 'the', 'head', 'of', 'the', 'local', 'Teams', '##ters', 'Union', '.', 'Finally', ',', 'ton', '##y', 'implica', '##ted', 'ja', '##ck', ',', 'an', 'East', 'Bat', '##on', 'Rouge', 'deputy', 'she', '##riff', ',', 'in', 'a', 'scheme', 'to', 'move', 'money', 'between', 'the', 'Teams', '##ters', 'Union', 'and', 'ton', '##y', '[UNK]', 's', 'political', 'opponent', '.', 'ja', '##ck', 'successfully', 'sue', '##d', 'ton', '##y', 'for', 'defa', '##mation', '.', 'Louisiana', '[UNK]', 's', 'First', 'Circuit', 'Court', 'of', 'Appeals', 'reverse', '##d', ',', 'holding', 'that', 'ja', '##ck', 'did', 'n

In [9]:
print(len(tokenized_texts))

length = []

one = []
two = []
tree = []
four = []
five = []
six = []
seven = []
eight = []
nine = []
ten = []

for i in tokenized_texts:
    length.append(len(i))

    if len(i) >= 0 and len(i) < 100:
        one.append(len(i))
    if len(i) >= 100 and len(i) < 200:
        two.append(len(i))
    if len(i) >= 200 and len(i) < 300:
        tree.append(len(i))
    if len(i) >= 300 and len(i) < 400:
        four.append(len(i))
    if len(i) >= 400 and len(i) < 500:
        five.append(len(i))
    if len(i) >= 500 and len(i) < 600:
        six.append(len(i))
    if len(i) >= 600 and len(i) < 700:
        seven.append(len(i))
    if len(i) >= 700 and len(i) < 800:
        eight.append(len(i))
    if len(i) >= 800 and len(i) < 900:
        nine.append(len(i))
    if len(i) >= 900 and len(i) < 1000:
        ten.append(len(i))

print(max(length))
print(min(length))
print(sum(length)/len(length))

print('================')
print(len(one))
print(len(two))
print(len(tree))
print(len(four))
print(len(five))
print(len(six))
print(len(seven))
print(len(eight))
print(len(nine))
print(len(ten))

100
613
45
250.42
6
24
45
16
5
3
1
0
0
0


In [10]:
# MAX_LEN = 500
MAX_LEN = 300
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([   101,  10576,  11120,  10365,    117,  10828,    117,  22464,
        10157,    117,    169,  22102,  10142,  11683,  14301,    117,
        11019,    169,  14162,  29811,  10106,  62949,  10263,  32721,
          117,  22886,    119,  12550,  10531,  29811,    117,  22464,
        10157,  37303,  10226,  13736,  53610,  10108,  11223,    169,
        33065,  10111,  10108,  11223,  16247,  10106,  30931,  19286,
        10169,  10105,  13578,  10108,  10105,  11436,  25754,  21891,
        11457,    119,  51857,    117,  22464,  10157,  54134,  11912,
        10201,  11263,    117,  10151,  11830,  62949,  10263,  32721,
        45404,  10833,  40738,    117,  10106,    169,  41209,  10114,
        18577,  17920,  10948,  10105,  25754,  21891,  11457,  10111,
        22464,  10157,    100,    187,  13736,  53610,    119,  10201,
        11263,  32094,  15893,  10162,  22464,  10157,  10142,  93279,
        41856,    119,  22886,    100,    187,  12128,  28418,  14100,
      

In [11]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [12]:
train_data_frame['first_party_winner'].values

array([1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train_data_frame['first_party_winner'].values, random_state=42, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=42, 
                                                       test_size=0.1)

In [14]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [15]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [16]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1080 Ti


In [17]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [18]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/home/moonstar/anaconda3/envs/pytorch/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [19]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [20]:
print(len(train_dataloader))

3


In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

train_loss = []
accuracy = []

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 1000 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    train_loss.append(avg_train_loss)
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    accuracy.append(eval_accuracy/nb_eval_steps)

print("")
print("Training complete!")